In [ ]:
# documentation for NASDAQ ITCH data: 
# http://www.nasdaqtrader.com/content/technicalsupport/specifications/dataproducts/NQTVITCHspecification.pdf

In [140]:
import pandas as pd
import numpy as np

In [ ]:
# for day in date_range : 
for ticker in nasdaq_tickers :
    itch_data = pd.read_csv(FOLDER + DATE + '_' + ticker + '.csv') # folder and date 
    

In [184]:
# check to make sure the ask and bid depth match the number of orders we have recorded
def verify_book(book, index, row) :
    side = book.groupby('side')
    bid_ask_depth = side.apply(lambda df: np.sum(df.shares))
    if (bid_ask_depth[0] != row['bid depth']) or (bid_ask_depth[1] != row['ask depth']) :
        raise Exception(f'Parsing error. Recorded orders don\'t match the data. Error occured on line {index}')
# this function worked well for debugging at the beginning, but later it becomes hard 
# to use because prices that were once ask prices can later become bid prices, making 
# ask and bid depth harder to compute

In [179]:
def init_book_shape(smallest_bin, exp_growth_factor, N) :
    bins = [-smallest_bin * (exp_growth_factor ** n) for n in reversed(range(N))] + [0] + [smallest_bin * (exp_growth_factor ** n) for n in range(N-1)]
    bins.append('shares')
    bins.append('price')
    book_shape = pd.DataFrame(columns=bins)
    bins.pop()
    bins.pop()
    bins = np.asarray(bins)
    return [book_shape, bins]

In [143]:
def get_bin(order_price, price) :
    cap = smallest_bin * (exp_growth_factor ** N-1)
    percent_dif = (order_price - price) / price
    if percent_dif <= cap :
        return np.where(percent_dif > bins)[0][-1]

In [144]:
itch_data = pd.read_csv('010416_CPN.csv')

/home/ndriggs/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [190]:
book = pd.DataFrame([[' B', 0, 0],[' S', 0, 0]], columns=['side','price','shares']) # initialize with zero asks and bids 
smallest_bin = 0.005
exp_growth_factor = 2 
N = 17
book_shape, bins = init_book_shape(smallest_bin, exp_growth_factor, N)

first_u = True # Order replace messages ('U' for update) come in pairs, the first being a 
               # deletion of previous order and the second adding a new order to the book

for index, row in itch_data.iterrows() :
    order_type = row['type'] 
    if order_type == 'P' :
        continue # Trade messages (non-cross) do not affect the book (see the docs)
    elif order_type == 'A' or order_type == 'F' or (order_type == 'U' and not first_u) : # adding an order to the book
        try :
            book.iloc[np.where(book['price'] == row['price'])[0][0], 2] += row['shares']
        except IndexError :
            book.loc[len(book)] = [row['side']] + [row['price']] + [row['shares']]
        if order_type == 'U' :
            first_u = True
    elif order_type == 'D' or order_type == 'X' or (order_type == 'U' and first_u): # deleting an order
        book.iloc[np.where(book['price'] == row['price'])[0][0], 2] -= row['shares']
        if order_type == 'U' :
            first_u = False
    elif order_type == 'E' or order_type == 'C': # order was executed
        try :
            last_price = book_shape.iloc[-1, -1] # price the last transaction was executed at
        except IndexError :
            last_price = row['price'] # at start of trading day there is no last transaction, use price of this transaction
        book_shape_temp = [0] * 2*N
        for i in range(len(book)) :
        # compute how far it is from the price
            byn = get_bin(book.loc[i, 'price'], last_price)
            if byn is not None :
                # add that many shares to its bin
                book_shape_temp[byn] += book.loc[i, 'shares']
        if row['side'] == ' S' :
            book_shape_temp.append(row['shares'])
        else :
            book_shape_temp.append(-1 * row['shares'])
        book_shape_temp.append(row['price']) # price that this trade executed at 
        new_book_shape_row = pd.DataFrame([book_shape_temp], columns = book_shape.columns)
        book_shape = pd.concat([book_shape, new_book_shape_row], ignore_index=True)
        book_shape.reset_index()
        # update book
        book.iloc[np.where(book['price'] == row['price'])[0][0], 2] -= row['shares']
    else :
        raise Exception(f'Incorrect order type. Order type was {order_type}')
    book.sort_values(by='price', inplace=True, ignore_index=True)
    if index == 10000 :
        break
    #verify_book(book, index, row) # check to make sure the ask and bid depth match the number of orders we have recorde

In [194]:
book_shape.iloc[30:50, 9:28]

,-0.64,-0.32,-0.16,-0.08,-0.04,-0.02,-0.01,-0.005,0,0.005,0.01,0.02,0.04,0.08,0.16,0.32,0.64,1.28,2.56
30,0,2900,100,12400,12400,2976,3400,1040,1300,1900,14476,0,12410,100,2900,60,0,0,0
31,0,2900,100,12400,12400,2976,3400,1268,1400,1900,14476,0,12410,100,2900,60,0,0,0
32,0,2900,100,12400,12900,2576,3300,1428,1500,1800,14276,0,12410,100,2900,60,0,0,0
33,0,2900,100,12400,12900,2576,3300,1368,1500,1800,14276,0,12410,100,2900,60,0,0,0
34,0,2900,100,12400,12900,2576,2600,2168,1300,1800,14276,0,12410,100,2900,60,0,0,0
35,0,2900,100,12400,12900,3476,1800,2168,1200,1800,14276,0,12410,100,2900,60,0,0,0
36,0,2900,100,12100,13200,3376,1800,2768,906,2000,14276,0,12410,100,2900,60,0,0,0
37,0,2900,100,12400,13400,2976,1800,2768,1100,2200,13976,0,12410,100,2900,60,0,0,0
38,0,2900,100,12400,12400,3276,2000,2268,1506,1800,14276,0,12410,100,2900,60,0,0,0
39,0,2900,100,12400,12400,3276,2000,2468,1500,1800,14276,0,12410,100,2900,60,0,0,0
